gdutils.qa
==============

``dataqa`` is a module in package ``gdutils`` that provides functions used in data quality assurance.

*Status:* In development

---

__Examples Setup__

The following commands are used for setting up the examples below. 

*Note:* The example input files were pulled and converted from the GeoJSON [link](http://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_110m_land.geojson) provided in the [geopandas IO docs](https://geopandas.org/io.html).

In [ ]:
# Install ``gdutils`` package
!pip install git+https://github.com/KeiferC/gdutils.git > /dev/null

In [ ]:
import gdutils.dataqa as dq # imports the ``dataqa`` module

import geopandas as gpd
import pandas as pd

---

Example 1. 
-------------------------------


__Example 1.1.__ 


- Example 1.1.1. 

In [ ]:
# Ex. 1.1.1



Example 2. 
-------------------------------------------------------------------------

__Example 2.1.__ 

In [ ]:
# Ex. 2.1


---

__Examples Cleanup__

The following commands are used to reset and clean up the examples above.

In [ ]:
# Remove outputs
!rm -r outputs

In [ ]:
# Uninstall Package
!echo y | pip uninstall gdutils

In [ ]:
# Reset Jupyter Notebook IPython Kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")